In [99]:
import json

from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import torch

model_path="ibm-granite/granite-3.2-2b-instruct"
device= "cuda" if torch.cuda.is_available() else ("mps" if torch.mps.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map=device,
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_path
)

model.save_pretrained("base_models/granite-3.2-2b-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [70]:
conv = [{"role": "user", "content":"Translate to english: കണ്ണടയും തൊപ്പിയും ധരിച്ച ഒരാളുടെ ക്ലോസ് അപ്പ്"}]

input_ids = tokenizer.apply_chat_template(conv, return_tensors="pt", thinking=False, return_dict=True, add_generation_prompt=True).to(device)
set_seed(42)
output = model.generate(
    **input_ids,
    max_new_tokens=8192,
)

prediction = tokenizer.decode(output[0, input_ids["input_ids"].shape[1]:], skip_special_tokens=True)
print(prediction)

A person's closet filled with both canes and crutches.


In [62]:
from datasets import Dataset

def preprocess_data(sources, destinations):
    dataset = [[{"role": "user", "content": f"Translate to english: {src}"} , {"role": "assistant", "content": dest}] for src, dest in zip(sources, destinations)]
    dataset = Dataset.from_dict({"formatted_chat": dataset})
    return dataset.map(lambda x: {"formatted_chat": tokenizer.apply_chat_template(x, tokenize=False, add_generation_prompt=False)})

def get_translation_loss(examples):
    input_ids = tokenizer.apply_chat_template(examples, return_tensors="pt", thinking=False, return_dict=True, add_generation_prompt=True).to(device)
    print(input_ids)
    

with open("data/coco.ml.txt", "r") as f:
    src_data = f.readlines()

with open("data/coco.en.txt", "r") as f:
    dest_data = f.readlines()

data = preprocess_data(src_data[:10], dest_data[:10])
data

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

UndefinedError: datasets.formatting.formatting.LazyRow object has no element 0

In [86]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.2-2b-instruct")

In [85]:
count, n = 0, 10000
for i in range(n):
    sample = src_data[i]
    count += tokenizer.decode(tokenizer.encode(sample)) == sample

count / n

0.9989

In [98]:
# Dataset preparation

# {
#     "messages": [
#         {"role": "system", "content": "Translate the Malayalam text to English."},
#         {"role": "user", "content": "മലയാളം ടെക്സ്റ്റ്"},
#         {"role": "assistant", "content": "Malayalam text"}
#     ]
# }

n = len(src_data)
dataset = []
for mal, eng in zip(src_data[:n], dest_data[:n]):
    sample = {
        "messages": [
            {"role": "system", "content": "Translate the Malayalam text to English."},
            {"role": "user", "content": mal.strip()},
            {"role": "assistant", "content": eng.strip()}
        ]
    }
    dataset.append(sample)

import json

with open("data/trans.jsonl", "w") as f:
    for sample in dataset:
        f.write(json.dumps(sample, ensure_ascii=False) + "\n")